In [1]:
try:
    from isaacgym import gymtorch
    from isaacgym import gymapi
    from bc_algos.dataset.isaac_gym import IsaacGymDataset
    from bc_algos.rollout.isaacgym_simple import IsaacGymSimpleRolloutEnv
except ImportError:
    pass
try:
    from bc_algos.dataset.robomimic import RobomimicDataset
    from bc_algos.rollout.robomimic import RobomimicRolloutEnv
except ImportError:
    pass
import bc_algos.utils.obs_utils as ObsUtils
import bc_algos.utils.tensor_utils as TensorUtils
from bc_algos.models.obs_nets import ObservationGroupEncoder, ActionDecoder
from bc_algos.models.backbone import Transformer, MLP
from bc_algos.models.policy_nets import BC_Transformer, BC, BC_MLP

import bc_algos.utils.constants as Constants
from torch.utils.data import DataLoader
from addict import Dict
import json

config_path = "../config/bc_transformer.json"
video_dir = "../outputs/test_rollout"

# load config 
with open(config_path, 'r') as f:
    config = json.load(f)
config = Dict(config)

Importing module 'gym_38' (/home/niksrid/IsaacGym/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/niksrid/IsaacGym/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.3.0
Device count 4
/home/niksrid/IsaacGym/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/niksrid/.cache/torch_extensions/py38_cu118 as PyTorch extensions root...
Emitting ninja build file /home/niksrid/.cache/torch_extensions/py38_cu118/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...
/home/niksrid/miniconda3/envs/rlgpu/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### ObsUtils Init

In [2]:
ObsUtils.init_obs_utils(config=config)
print(ObsUtils.MODALITY_TO_ENC_CORE_CLASS)
print(ObsUtils.OBS_KEY_TO_SHAPE)
print(ObsUtils.OBS_KEY_TO_MODALITY)
print(ObsUtils.OBS_GROUP_TO_KEY)

OrderedDict([(<Modality.LOW_DIM: 'low_dim'>, <class 'bc_algos.models.obs_core.LowDimCore'>), (<Modality.RGB: 'rgb'>, <class 'bc_algos.models.obs_core.ResNet18Core'>)])
OrderedDict([('robot0_eef_pos', [3]), ('robot0_eef_quat', [4]), ('agentview_image', [3, 256, 256])])
OrderedDict([('robot0_eef_pos', 'low_dim'), ('robot0_eef_quat', 'low_dim'), ('agentview_image', 'rgb')])
OrderedDict([('obs', ['robot0_eef_pos', 'robot0_eef_quat', 'agentview_image'])])


/home/niksrid/miniconda3/envs/rlgpu/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/niksrid/miniconda3/envs/rlgpu/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Test Factory Functions

In [3]:
if config.dataset.type == Constants.DatasetType.ROBOMIMIC:
    trainset = RobomimicDataset.factory(config=config, train=True)
    validset = RobomimicDataset.factory(config=config, train=False)
elif config.dataset.type == Constants.DatasetType.ISAAC_GYM:
    trainset = IsaacGymDataset.factory(config=config, train=True)
    validset = IsaacGymDataset.factory(config=config, train=False)

obs_group_enc = ObservationGroupEncoder.factory(config=config)

if config.policy.type == Constants.PolicyType.MLP:
    backbone = MLP.factory(config=config, embed_dim=obs_group_enc.output_dim)
elif config.policy.type == Constants.PolicyType.TRANSFORMER:
    backbone = Transformer.factory(config=config)

action_dec = ActionDecoder.factory(config=config, input_dim=backbone.output_dim)

if config.rollout.type == Constants.RolloutType.ROBOMIMIC:
    rollout_env = RobomimicRolloutEnv.factory(
        config=config, 
        validset=validset, 
        normalization_stats=trainset.normalization_stats,
    )
elif config.rollout.type == Constants.RolloutType.ISAAC_GYM:
    rollout_env = IsaacGymSimpleRolloutEnv.factory(
        config=config, 
        validset=validset, 
        normalization_stats=trainset.normalization_stats,
    )

NameError: name 'RobomimicDataset' is not defined

###  Test Policy

In [ ]:
train_loader = DataLoader(trainset, batch_size=config.train.batch_size, shuffle=True)
train_loader_iter = iter(train_loader)
input = next(train_loader_iter)
input = BC.prepare_input(input=input)
input["obs"] = TensorUtils.slice(x=input["obs"], dim=1, start=0, end=config.dataset.frame_stack+1)

In [ ]:
if config.policy.type == Constants.PolicyType.MLP:
    policy = BC_MLP(obs_group_enc=obs_group_enc, backbone=backbone, action_dec=action_dec)
elif config.policy.type == Constants.PolicyType.TRANSFORMER:
    policy = BC_Transformer.factory(
        config=config, 
        obs_group_enc=obs_group_enc, 
        backbone=backbone, 
        action_dec=action_dec
    )

actions = policy(input)
assert list(actions.shape) == [config.train.batch_size, config.dataset.seq_length, *config.policy.action_shape]

### Test Rollout

In [ ]:
results = rollout_env.rollout_with_stats(
    policy=policy,
    demo_id=validset.demos[0],
    video_dir=video_dir,
    horizon=25,
    video_skip=1,
)
assert results["horizon"] == 25

### ObsUtils Deinit

In [ ]:
ObsUtils.deinit_obs_utils()